In [19]:
# Limpieza de la base de datos
import pandas as pd
import numpy as np

df = pd.read_csv(  # Lectura de datos
    "../data/BBDD Población Chile (1b).xlsx - Tabla ajustada población .csv",
    index_col=0
)


df = df.T  # Transposicion

# notacion europea a float
for col in df.columns:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )

df = df.apply(pd.to_numeric, errors="ignore")


df = df.replace(["NaN", "nan", "NAN", "None", ""], np.nan)

df = df.fillna(0)

print(df)

df.to_csv("../data/output.csv")

      Población total de jovenes (0-14)  Población total de adultos (15-64)  \
2002                          4156812.0                          10288683.0   
2003                          4116188.0                          10456445.0   
2004                          4057629.0                          10640163.0   
2005                          3991476.0                          10826184.0   
2006                          3921521.0                          11017529.0   
2007                          3864328.0                          11200341.0   
2008                          3823083.0                          11375175.0   
2009                          3792259.0                          11541067.0   
2010                          3767767.0                          11697176.0   
2011                          3748933.0                          11851404.0   
2012                          3732105.0                          11998670.0   
2013                          3712426.0             

/var/folders/ks/j310rx014qx5jxxp12t_qrqw0000gn/T/ipykernel_82294/1530326135.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors="ignore")


In [21]:
#regresion


df["tasa_rel_jovenes"] = df["dJ"] / df["Población total de jovenes (0-14)"]
df["tasa_rel_adultos"] = df["dA"] / df["Población total de adultos (15-64)"]
df["tasa_rel_mayores"] = df["dM"] / df["Poblacion total mayores (65+ años)"]

df[["tasa_rel_jovenes", "tasa_rel_adultos", "tasa_rel_mayores"]].head()

import statsmodels.api as sm

df["Año"] = df.index + 2002

X = sm.add_constant(df["Año"])
Y = df["tasa_rel_jovenes"]

modelo_mayores = sm.OLS(Y, X).fit()
print(modelo_mayores.summary())

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [ ]:
print(df.head)

Index(['Población total de jovenes (0-14)',
       'Población total de adultos (15-64)',
       'Poblacion total mayores (65+ años)', 'dJ', 'dA', 'dM'],
      dtype='object')
